## Import the package 

We start by importing the `siibra` package and other necessary libraries.

In [ ]:
import siibra
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nilearn import plotting, image

### Download an "fMRI" image

In [ ]:
url = "https://github.com/FZJ-INM1-BDA/siibra-tutorials/raw/refs/heads/incf_summerschool_2025/workshops/test_fmri.nii.gz"
img = siibra.retrieval.HttpRequest(url).get()
img.to_filename("./test_fmri.nii.gz")

## I. Integrate your image to siibra for local use

### 1. Create json configuration

In [ ]:
fmri_conf = {
    "@type": "siibra/feature/voi/v0.1",
    "name": "test fMRI feature",
    "modality": "fMRI",
    "space": {"name": "2009c nonlinear asym"},
    "providers": {"nii": "./test_fmri.nii.gz"},
}
with open("./fmri_conf_test.json", "wt") as fp:
    json.dump(fmri_conf, fp=fp, indent="\t")

### 2. Read the json configuration to test

In [ ]:
fmri_feat = siibra.from_json("./fmri_conf_test.json")
print(fmri_feat.name)
print(fmri_feat.modality)

In [ ]:
print(fmri_feat.get_boundingbox())

In [ ]:
img = fmri_feat.fetch()
plotting.plot_glass_brain(img, cmap='RdBu', symmetric_cbar=True)

### 3. Split input volume into cluster components
There are many ways to get components out of a feature map. Here we use siibra to
- draw random points from the distribution encoded by the input volume, then
 cluster them using DBSCAN, and
- build clusterwise featuremaps as Kernel Density estimates thereof.

(In this example, this more or less inverts the composition of the input volume from
the DiFuMo maps, but the idea for a general input image is to separate it into
components that have more meaningful correlations with brain regions than the full
image, which is usually a mixture distribution.)

In [ ]:
# set random seeed for reproducibility
np.random.seed(25)
N = 1000  # number of random samples
# drawing the samples results in a siibra PointCloud,
# which has reference space attached and can model point uncertainties.
samples = fmri_feat.draw_samples(N, e=5, sigma_mm=3)

# finding the clusters will result in a labelling of the point set.
samples.find_clusters(min_fraction=1 / 300, max_fraction=1 / 2)
clusterlabels = set(samples.labels) - {-1}

# Let's have a look at the clustered pointcloud
view = plotting.plot_glass_brain(
    fmri_feat.fetch(), alpha=1, threshold=15, cmap="RdGy"
)
view.add_markers(
    np.array(samples.as_list())[samples.labels >= 0],
    marker_size=5,
    marker_color=[samples.label_colors[lb] for lb in samples.labels if lb >= 0],
)

### 4. Select cluster and create a volume

In [ ]:
cl = 3
clustermap = siibra.volumes.from_pointcloud(samples, label=cl, target=fmri_feat)
plotting.plot_glass_brain(
    clustermap.fetch(),
    alpha=1,
    cmap="RdBu",
    title=f"Cluster #{cl}",
    symmetric_cbar=True,
)

### 5. Query features with the selected cluster

In [ ]:
genes = ["gabarapl1", "gabarapl2", "maoa", "tac1"]
gene_expressions = siibra.features.get(clustermap, "gene expressions", gene=genes)[0]
print("Found measurements:", len(gene_expressions.anchor.location))
gene_expressions.plot()

In [ ]:
gene_expressions.data

In [ ]:
gene_expressions.plot(by=["gene", "gender"])

### HANDS ON
Query for BigBrain Intensity Profiles and plot the first result

In [ ]:
siibra.features.get(clustermap, siibra.features.cellular.BigBrainIntensityProfile)[0].plot()

### HANDS ON
Plot one of the profiles

### HANDS ON
Query for Layerwise BigBrain Intensities and plot the first result

### HANDS ON
Query for MRI images and extract "CHENONCEAU: post mortem anatomical UHF MRI - 150um" high resolution image corresponding to the boundind box of the cluster

## II. Assign regions overlapping with the selected cluster
To assign the clusters to brain regions, we build feature maps from each cluster
and assign them to the Julich-Brain probabilistic maps. The assignment is one-to-many
since the structures in the image and parcellation are continuous. Assignments
report correlation, intersection over union, and some other measures which we can use
to filter and sort them. The result is an assignment table from cluster components
in the input volume to regions in the Julich-Brain atlas.

### 1. Select map to assign

In [ ]:
julich_pmaps = siibra.get_map(
    parcellation="julich 3.0.3", space="mni152", maptype="statistical"
)

In [ ]:
assignments = julich_pmaps.assign(clustermap, split_components=False)
print("Intersections found:", len(assignments))
assignments.sort_values("map value", ascending=False).head()

### 2. Filter assignments

In [ ]:
min_correlation = 0.2
min_map_value = 0.5
assignments.query(f"correlation >= {min_correlation}", engine="python", inplace=True)
assignments

### 3. Compare the cluster tp selected region

In [ ]:
selected_region = siibra.get_region("julich 3.0.3", "Area hOc1 (V1, 17, CalcS) left")

In [ ]:
display = plotting.plot_glass_brain(
    selected_region.get_regional_mask('mni152').fetch()
)
display.add_overlay(clustermap.fetch(),
    alpha=0.5,
    cmap="jet",
)

### HANDS ON
Find gene expresssions and compare to the query with the volume